In [1]:
import pandas as pd
from sqlalchemy import create_engine
from time import time
from decouple import config

In [ ]:
DATABASE_URL=config("DATABASE_URL")

In [13]:
df = pd.read_csv('green_tripdata_2019-01.csv.gz', nrows=100)

In [14]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.00,3.0,0.5,0.5,0.00,0.0,NaN,0.3,4.30,2,1,NaN
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,0.5,0.5,0.00,0.0,NaN,0.3,7.30,2,1,NaN
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,0.5,0.5,0.00,0.0,NaN,0.3,5.80,1,1,NaN
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,0.5,0.5,2.96,0.0,NaN,0.3,19.71,1,1,NaN
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,0.5,0.5,0.00,0.0,NaN,0.3,19.30,2,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,2019-01-01 00:15:42,2019-01-01 00:41:04,N,1,97,179,1,7.83,26.0,0.5,0.5,5.46,0.0,NaN,0.3,32.76,1,1,NaN
96,2,2019-01-01 00:21:28,2019-01-01 00:48:15,N,1,7,230,1,4.49,20.0,0.5,0.5,4.00,0.0,NaN,0.3,25.30,1,1,NaN
97,2,2019-01-01 00:17:10,2019-01-01 00:23:34,N,1,42,42,1,1.08,6.5,0.5,0.5,0.00,0.0,NaN,0.3,7.80,2,1,NaN
98,2,2019-01-01 00:25:01,2019-01-01 00:43:55,N,1,42,232,2,8.74,25.0,0.5,0.5,0.00,0.0,NaN,0.3,26.30,2,1,NaN


In [12]:
df.dtypes

VendorID                   int64
tpep_pickup_datetime      object
tpep_dropoff_datetime     object
passenger_count            int64
trip_distance            float64
RatecodeID                 int64
store_and_fwd_flag        object
PULocationID               int64
DOLocationID               int64
payment_type               int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtype: object

In [14]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

In [15]:
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

In [16]:
engine = create_engine(DATABASE_URL)
engine.connect()

In [17]:
print(pd.io.sql.get_schema(df, name='yellow_taxi', con=engine))


CREATE TABLE yellow_taxi (
	"VendorID" BIGINT, 
	tpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	tpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	"RatecodeID" BIGINT, 
	store_and_fwd_flag TEXT, 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	payment_type BIGINT, 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	congestion_surcharge FLOAT(53)
)




In [25]:
%time df.head(0).to_sql(name='yellow_taxi', con=engine, if_exists='replace')

CPU times: total: 46.9 ms
Wall time: 611 ms


In [28]:
with pd.read_csv('yellow_tripdata_2021-01.csv.gz', chunksize=100000) as iter_df:
    for df_chunk in iter_df:

        t_start = time()

        df_chunk.tpep_pickup_datetime = pd.to_datetime(df_chunk.tpep_pickup_datetime)
        df_chunk.tpep_dropoff_datetime = pd.to_datetime(df_chunk.tpep_dropoff_datetime)

        df_chunk.to_sql(name='yellow_taxi', con=engine, if_exists='append')

        t_end = time()

        print(f'chunk done in {t_end-t_start} seconds')
    
    print('all done!')

chunk done in 10.779372692108154
chunk done in 15.598631858825684
chunk done in 9.658520460128784
chunk done in 12.541609048843384
chunk done in 12.173084259033203
chunk done in 9.791495561599731
chunk done in 9.964760780334473
chunk done in 11.781901359558105
chunk done in 9.932461977005005
chunk done in 9.18806004524231
chunk done in 8.787911891937256
chunk done in 9.721214056015015


c:\Users\Qesh\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3191: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


chunk done in 8.739499807357788
chunk done in 5.943880319595337
